# Imports

In [3]:
import re
import string

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 150)

from preprocessing import tokenize

import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer

In [4]:
DATA_PATH_PREP = '../DATA/prepared'

# Load data

In [32]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Create samples

In [33]:
df_group_by_author = df_full.groupby("author")["text"].agg(" ".join).reset_index()
df_group_by_author

,author,text
0,aleko-konstantinov,"\n\n\n\tНашият помощник на регистратора е страшен комик. Изпокапваме от смях, когато почне да бомбардира несправедливата съдба.\n\t— Е, дявол да г..."
1,dimityr-dimov,"\n\n\n\tI глава\n\n\tГроздоберът беше към края си.\n\tОт малките вили и кирпичените постройки сред лозята се разнасяха ту задружни песни, бързи и ..."
2,dimityr-talev,"\n\n\n\tПърва част\n\tХаджи Серафимовата внука\n\n\n\n\tОвде дърво столовито,\n\tстоловито, грановито,\n\tгранки му са до небеси,\n\tа корени — су..."
3,elin-pelin,"\n\n\n\tПърва глава\n\n\tНай-заможният човек в селото беше дядо Йордан Геракът. Пъргав и трудолюбив, той бе работил през целия си живот и бе сполу..."
4,ivan_vazov,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
5,jordan-jovkov,"\n\n\n\tI\n\n\tГрадът бързо се подновяваше и растеше. Но все пак оставаха много улици, където още можеше да се видят тихи, едновремешни къщурки, с..."


# Encode sentences

# Create samples

In [35]:
limit_x = df_group_by_author['text'].map(lambda t: len(t.strip().split())).min() - 1000
print(f'{limit_x=}')
limit_y = 1000

num_authors = df_group_by_author.shape[0]
num_samples = 100

start_idxs = np.random.randint(0, limit_x, (num_authors, num_samples))
sample_lengths = np.random.randint(3, limit_y, (num_authors, num_samples))

df_samples = pd.DataFrame(columns=['text', 'author'])
for i in range(num_authors):
    author_name = df_group_by_author['author'][i]
    for j, start_idx in enumerate(start_idxs[i]):
        sample = df_group_by_author['text'][i][start_idx : (start_idx + sample_lengths[i, j])]
        df_samples.loc[len(df_samples)] = [sample, author_name]
df_samples

limit_x=29374


,text,author
0,"ай Ганьо я оскърбил с действие, похванал я и не само я похванал, ами… Тя искаше да вика полицейския. Скандал!\n\t— Скоро да се махаш от",aleko-konstantinov
1,"они предложението ми, защото бил чел доста на времето, и счете за по-практично да си дремне. Защо ще стои бадева буден; тъй и тъй е дал пари за же...",aleko-konstantinov
2,миризма силна — ще бръкне да извади някое шише — иди го г,aleko-konstantinov
3,"чките жили на моето тяло, нека се стопли това тяло, което се сгърчваше и вкоченясваше ето вече третя година под хладния меч на Парламентарната анк...",aleko-konstantinov
4,зовото масл,aleko-konstantinov
...,...,...
595,"беше Леопарди и към неговата мрачна и меланхолична поезия той показваше голяма слабост. Неговото име беше на устните му, негови стихове цитираше....",jordan-jovkov
596,"кой знае защо не успяваше поне толкоз, колкото и сам той, и другите очакваха. Запазен е и досега един особен документ от тая тъй известна история....",jordan-jovkov
597,"че и дрехите му се окъсваха дотолкоз, че приличаха на старо някое знаме.\n\tЗа по-късо или по-дълго време, благодетелите му се отказваха от него и...",jordan-jovkov
598,"сивели дъски не една влюбена двойка беше издълбала инициалите си.\n\tИмаше също тъй и хора, толкоз много познати, толкоз неизменни и някак старинн...",jordan-jovkov


# Encode sentences

In [36]:
model_bg = SentenceTransformer('distiluse-base-multilingual-cased-v2')

df_samples['text'] = df_samples['text'].map(lambda txt: model_bg.encode(sent_tokenize(txt.strip())))
df_samples

,text,author
0,"[[0.023406774, 0.08853289, 0.017080262, 0.01902649, -0.000730425, -0.029901007, -0.00984154, 0.026202017, 0.0070298235, 0.03409542, 0.033951174, 0...",aleko-konstantinov
1,"[[0.008410395, 0.067847416, -0.0007493276, 0.005137028, -0.012316157, -0.036057256, -0.0023128237, 0.023626203, -0.017832518, -0.009246053, 0.0099...",aleko-konstantinov
2,"[[0.009139298, -0.0103367055, 0.0049481294, -0.0156199, 0.06459348, 0.014258349, 0.004487975, -0.0065842653, 0.021060249, 0.021272501, -0.05227449...",aleko-konstantinov
3,"[[-0.029517908, 0.040320795, 0.087215476, 0.004826014, 0.0013982328, -0.04777546, 0.064770505, 0.02235388, 0.049564574, -0.018345784, 0.05281924, ...",aleko-konstantinov
4,"[[0.00069873856, 0.02079982, -0.0075297942, 0.008134051, -0.0024792342, -0.00024178345, -0.013637532, -0.045004964, 0.019960841, 0.0067607923, -0....",aleko-konstantinov
...,...,...
595,"[[-0.048833206, -0.015362595, -0.0049023125, 0.0062374333, -0.02321151, 0.076304376, -0.05231935, -0.04608202, -0.04466921, 0.018332852, 0.0328529...",jordan-jovkov
596,"[[-0.027542586, -0.011782025, -0.004671761, -0.027033495, 0.011562013, -0.017644467, 0.04879399, -0.006814167, 0.020293508, -0.05600276, -0.008938...",jordan-jovkov
597,"[[-0.032206114, -0.014206536, -0.011887201, -0.012597629, -0.03612469, 0.045266483, 0.015761884, 0.02236377, 0.007701067, 0.022063926, 0.002091417...",jordan-jovkov
598,"[[0.033862233, -0.012871197, 0.017728139, -0.009554414, -0.05449718, -0.021147178, 0.012414615, 0.017541569, -0.01892361, 0.027987579, 0.03249506,...",jordan-jovkov


In [50]:
df_samples['text'] = df_samples['text'].map(lambda arr: arr.reshape(-1))
df_samples

,text,author
0,"[0.023406774, 0.08853289, 0.017080262, 0.01902649, -0.000730425, -0.029901007, -0.00984154, 0.026202017, 0.0070298235, 0.03409542, 0.033951174, 0....",aleko-konstantinov
1,"[0.008410395, 0.067847416, -0.0007493276, 0.005137028, -0.012316157, -0.036057256, -0.0023128237, 0.023626203, -0.017832518, -0.009246053, 0.00995...",aleko-konstantinov
2,"[0.009139298, -0.0103367055, 0.0049481294, -0.0156199, 0.06459348, 0.014258349, 0.004487975, -0.0065842653, 0.021060249, 0.021272501, -0.052274495...",aleko-konstantinov
3,"[-0.029517908, 0.040320795, 0.087215476, 0.004826014, 0.0013982328, -0.04777546, 0.064770505, 0.02235388, 0.049564574, -0.018345784, 0.05281924, -...",aleko-konstantinov
4,"[0.00069873856, 0.02079982, -0.0075297942, 0.008134051, -0.0024792342, -0.00024178345, -0.013637532, -0.045004964, 0.019960841, 0.0067607923, -0.0...",aleko-konstantinov
...,...,...
595,"[-0.048833206, -0.015362595, -0.0049023125, 0.0062374333, -0.02321151, 0.076304376, -0.05231935, -0.04608202, -0.04466921, 0.018332852, 0.0328529,...",jordan-jovkov
596,"[-0.027542586, -0.011782025, -0.004671761, -0.027033495, 0.011562013, -0.017644467, 0.04879399, -0.006814167, 0.020293508, -0.05600276, -0.0089383...",jordan-jovkov
597,"[-0.032206114, -0.014206536, -0.011887201, -0.012597629, -0.03612469, 0.045266483, 0.015761884, 0.02236377, 0.007701067, 0.022063926, 0.0020914173...",jordan-jovkov
598,"[0.033862233, -0.012871197, 0.017728139, -0.009554414, -0.05449718, -0.021147178, 0.012414615, 0.017541569, -0.01892361, 0.027987579, 0.03249506, ...",jordan-jovkov


In [20]:
unfolded = pd.DataFrame(df_samples['text'].tolist()).fillna(0)
unfolded['author'] = df_samples['author']
unfolded

,0,1,2,3,4,5,6,7,8,9,...,13303,13304,13305,13306,13307,13308,13309,13310,13311,author
0,0.023407,0.088533,0.017080,0.019026,-0.000730,-0.029901,-0.009842,0.026202,0.007030,0.034095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,aleko-konstantinov
1,0.008410,0.067847,-0.000749,0.005137,-0.012316,-0.036057,-0.002313,0.023626,-0.017833,-0.009246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,aleko-konstantinov
2,0.009139,-0.010337,0.004948,-0.015620,0.064593,0.014258,0.004488,-0.006584,0.021060,0.021273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,aleko-konstantinov
3,-0.029518,0.040321,0.087215,0.004826,0.001398,-0.047775,0.064771,0.022354,0.049565,-0.018346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,aleko-konstantinov
4,0.000699,0.020800,-0.007530,0.008134,-0.002479,-0.000242,-0.013638,-0.045005,0.019961,0.006761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,aleko-konstantinov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.048833,-0.015363,-0.004902,0.006237,-0.023212,0.076304,-0.052319,-0.046082,-0.044669,0.018333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jordan-jovkov
596,-0.027543,-0.011782,-0.004672,-0.027033,0.011562,-0.017644,0.048794,-0.006814,0.020294,-0.056003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jordan-jovkov
597,-0.032206,-0.014207,-0.011887,-0.012598,-0.036125,0.045266,0.015762,0.022364,0.007701,0.022064,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jordan-jovkov
598,0.033862,-0.012871,0.017728,-0.009554,-0.054497,-0.021147,0.012415,0.017542,-0.018924,0.027988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jordan-jovkov


# Saving to files

In [22]:
unfolded.to_pickle(f'{DATA_PATH_PREP}/03_df_samples_sbert.pkl')